## Exploring the Walmart Amazon dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
%env OPENAI_API_KEY=

env: OPENAI_API_KEY=


#### Install the autolabel library

In [34]:
!pip3 install 'refuel-autolabel[openai]'

zsh:1: no matches found: refuel-autolabel[openai]


## Run the model

In [21]:
from autolabel import LabelingAgent
import json

In [25]:
with open('config_walmart_amazon.json', 'r') as f:
     config = json.load(f)

In [26]:
agent = LabelingAgent(config=config)

In [27]:
agent.plan('test.csv')

2023-06-12 13:29:49 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬──────────┐
│ Total Estimated Cost     │ $6.685   │
│ Number of Examples       │ 1998     │
│ Average cost per example │ $0.00335 │
└──────────────────────────┴──────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange_charge
exchange_rate
exchange_via_app
extra_charge_on_statement
failed_transfer
fiat_currency_support
get_disposable_

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [28]:
labels, df, metrics_list = agent.run('test.csv', max_items=100)

2023-06-12 13:30:13.878 | INFO     | autolabel.labeler:run:141 - Task run already exists.


There is an existing task with following details: id='2287622123' created_at=datetime.datetime(2023, 6, 12, 12, 44,
38, 598625) task_id='ab16ad85a971805380e6a24713f197f6' dataset_id='1d12283d38845b25982ede8f1d7092ae' 
current_index=5 output_file='banking_test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None

Evaluating the existing task...

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 5       │ -inf      │ 0.6      │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

5 examples have been labeled so far.

...

y


/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Actual Cost: 0.0


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 5       │ -inf      │ 0.6      │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

### Compute confidence scores


In [29]:
config["model"]["compute_confidence"] = True

In [30]:
agent = LabelingAgent(config=config)

In [31]:

agent.plan('test.csv')


/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬──────────┐
│ Total Estimated Cost     │ $6.684   │
│ Number of Examples       │ 1998     │
│ Average cost per example │ $0.00335 │
└──────────────────────────┴──────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange_charge
exchange_rate
exchange_via_app
extra_charge_on_statement
failed_transfer
fiat_currency_support
get_disposable_

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [32]:
labels, df, metrics_list = agent.run('test.csv', max_items=100)

2023-06-12 13:32:29.275 | INFO     | autolabel.labeler:run:141 - Task run already exists.


There is an existing task with following details: id='2710642923' created_at=datetime.datetime(2023, 6, 12, 12, 45,
14, 233284) task_id='b5f154b0fcd26fd7c4fbac6ea5beca8d' dataset_id='1d12283d38845b25982ede8f1d7092ae' 
current_index=5 output_file='banking_test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None

Evaluating the existing task...

Metric: auroc: 1.0


┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold          ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 5       │ -inf               │ 0.6      │ 1.0             │
│ 0       │ 3.715415506885223  │ 0.0      │ 0.0             │
│ 1       │ 2.715415506885223  │ 1.0      │ 0.2             │
│ 3       │ 2.6973048594667963 │ 1.0      │ 0.6             │
│ 5       │ 2.3766050567851615 │ 0.6      │ 1.0             │
└─────────┴────────────────────┴──────────┴─────────────────┘

...

y


/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Metric: auroc: 1.0
Actual Cost: 0.0


┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold          ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 5       │ -inf               │ 0.6      │ 1.0             │
│ 0       │ 3.715415506885223  │ 0.0      │ 0.0             │
│ 1       │ 2.715415506885223  │ 1.0      │ 0.2             │
│ 3       │ 2.6973048594667963 │ 1.0      │ 0.6             │
│ 5       │ 2.3766050567851615 │ 0.6      │ 1.0             │
└─────────┴────────────────────┴──────────┴─────────────────┘

Total number of failures: 0